In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import datetime
from scipy import stats

import seaborn as sns
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'

In [2]:
full_df=pd.read_pickle('../datasets/df_userID.pickle')

In [3]:
full_df=full_df[~full_df.Weather.isnull()]

In [4]:
full_df.head()

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,UserID
534409,"Zermatt Marathon, Zermatt",sam. 05.07.2008,2008,7,M45,42.195,Abgottspon Helmut,M,1962.0,Staldenried,165,06:10:37.200000,00:08:47,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...,Abgottspon Helmut 1962.0
534410,"Zermatt Marathon, Zermatt",sam. 05.07.2008,2008,7,F55,42.195,Abgottspon Julia,F,1950.0,Thun,5,05:34:08.200000,00:07:55,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...,Abgottspon Julia 1950.0
534411,"Zermatt Marathon, Zermatt",sam. 05.07.2008,2008,7,M65,42.195,Abgottspon Medard,M,1942.0,Visp,2,05:13:54.700000,00:07:26,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...,Abgottspon Medard 1942.0
534412,"Zermatt Marathon, Zermatt",sam. 05.07.2008,2008,7,M50,42.195,Achterberg Nico,M,1957.0,Veenendaal,80,05:38:35,00:08:01,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...,Achterberg Nico 1957.0
534413,"Zermatt Marathon, Zermatt",sam. 05.07.2008,2008,7,M50,42.195,Ackeret Peter,M,1957.0,Deitingen,11,04:26:03.500000,00:06:18,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...,Ackeret Peter 1957.0


In [5]:
full_df.Race=full_df.Race.apply(lambda x: 'Lucerne Marathon, Luzern' if x=='SwissCityMarathon - Lucerne, Luzern' else x)

# Categorize weather

In [ ]:
full_df.Weather.value_counts().index

2 -> Sunny
1 -> Cloudy
0 -> bad weather

In [90]:
dict_favorable_weather={'Partly cloudy':1, 'Clear':2, 'Sunny':2, 'Mist':0, 'Patchy rain possible':1,
       'Cloudy':1, 'Light rain':0, 'Light rain shower':0, 'Moderate rain':0,
       'Overcast':1, 'Fog':0, 'Moderate or heavy rain shower':0,
       'Moderate rain at times':1, 'Light drizzle':0, 'Light snow':0,
       'Patchy light drizzle':0, 'Moderate snow':0, 'Patchy light rain':0,
       'Patchy light rain with thunder':0, 'Freezing fog':0, 'Heavy snow':0,
       'Heavy rain':0, 'Moderate or heavy rain with thunder':0,
       'Moderate or heavy sleet':0, 'Patchy moderate snow':0,
       'Heavy rain at times':0, 'Light sleet':0,
       'Moderate or heavy snow with thunder':0}

In [ ]:
def categorize_weather(x):
    return dict_favorable_weather[x]

In [ ]:
full_df.Weather=full_df.Weather.apply(categorize_weather)

# Races with a lot of years

In [21]:
good_distances=full_df[(full_df.Distance==10)]# | (full_df.Distance==21.0975) | (full_df.Distance==42.195)]

In [22]:
a=good_distances.groupby([good_distances.Race,good_distances.RaceYear]).count().reset_index()

In [23]:
a=a[['Race','RaceYear','Date']]
a=a.rename(columns={'Date':'Runners'})
a['Years']=1
a

,Race,RaceYear,Runners,Years
0,10km Schweizer Meisterschaften Lyss,2011,460,1
1,10km Schweizer Meisterschaften Lyss,2012,457,1
2,10km de Payerne,2009,368,1
3,10km de Payerne,2010,468,1
4,"10km, Payerne",2011,212,1
5,"10km, Payerne",2012,455,1
6,20km de Lausanne,2009,4789,1
7,20km de Lausanne,2010,5197,1
8,20km de Lausanne,2011,5823,1
9,20km de Lausanne,2012,6052,1


In [24]:
race_stats=a.groupby(a.Race).sum()

In [25]:
race_stats

,RaceYear,Runners,Years
Race,,,
10km Schweizer Meisterschaften Lyss,4023,917,2
10km de Payerne,4019,836,2
"10km, Payerne",4023,667,2
20km de Lausanne,14084,45029,7
A travers Carouge - Run to Run,8038,1734,4
BCN Tour La Brévine,2011,1004,1
"Birslauf (Herbst), Basel",4017,939,2
Birslauf Basel,8052,2320,4
"Brienzerseelauf, Bönigen",2015,63,1


In [36]:
race_stats=race_stats[race_stats.Years>5]
race_stats['RunnersPerYear']=race_stats.Runners/race_stats.Years
race_stats=race_stats[race_stats.RunnersPerYear>500]
race_stats

,RaceYear,Runners,Years,RunnersPerYear
Race,,,,
20km de Lausanne,14084,45029,7,6432.714286
"Hallwilerseelauf, Beinwil am See",14084,7689,7,1098.428571
Lausanne Marathon,12075,13044,6,2174.000000
"Rotseelauf, Ebikon",12069,5339,6,889.833333
Schweizer Frauenlauf Bern,14084,25398,7,3628.285714
"Stralugano, Lugano",14084,12074,7,1724.857143
"Thuner Stadtlauf, Thun",14078,5141,7,734.428571


In [37]:
good_games=race_stats.index
good_games

Index(['20km de Lausanne', 'Hallwilerseelauf, Beinwil am See',
       'Lausanne Marathon', 'Rotseelauf, Ebikon', 'Schweizer Frauenlauf Bern',
       'Stralugano, Lugano', 'Thuner Stadtlauf, Thun'],
      dtype='object', name='Race')

# Example with Lausanne Marathon

In [38]:
reduced_df=full_df[full_df.Race.isin(good_games)]

In [39]:
reduced_df.shape

(254969, 19)

In [40]:
reduced_df.RaceYear.value_counts()

2015    44202
2014    42637
2013    38492
2011    34536
2012    34480
2010    32342
2009    26909
2008     1371
Name: RaceYear, dtype: int64

In [41]:
km10=reduced_df[reduced_df.Distance==10]

In [44]:
km10.Race.value_counts()

20km de Lausanne                    45029
Schweizer Frauenlauf Bern           25398
Lausanne Marathon                   13044
Stralugano, Lugano                  12074
Hallwilerseelauf, Beinwil am See     7689
Rotseelauf, Ebikon                   5339
Thuner Stadtlauf, Thun               5141
Name: Race, dtype: int64

In [46]:
km10.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance', 'Name',
       'Sex', 'Year', 'LivingPlace', 'Rank', 'Time', 'Pace', 'Place',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'UserID'],
      dtype='object')

In [47]:
km10=km10[['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance',
       'Sex', 'Time', 'Pace',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'UserID']]

In [52]:
def add_mean_std_race(x):
    x['MeanRace']=np.mean(x.Time)
    x['StdRace']=np.std(x.Time)/np.sqrt(x.shape[0])
    x['Count']=x.shape[0]
    return x

In [53]:
def add_mean_multiple_years(x):
    x['MeanMultipleYears']=np.mean(x.Time)
    return x

In [63]:
km10=km10.groupby(km10.RaceID).apply(add_mean_std_race)
km10=km10.groupby(km10.Race).apply(add_mean_multiple_years)
km10=km10[km10.Sex=='M']

In [72]:
km10['DiffFromGlobalMean']=km10.MeanRace-km10.MeanMultipleYears
km10.DiffFromGlobalMean=km10.DiffFromGlobalMean.apply(lambda x: x.total_seconds())

In [74]:
km10['AvgTemp']=(km10.MinTemp+km10.MaxTemp)/2

In [73]:
km10

,Race,Date,RaceYear,RaceMonth,Category,Distance,Sex,Time,Pace,MinTemp,MaxTemp,Weather,RaceID,UserID,MeanRace,StdRace,Count,MeanMultipleYears,DiffFromGlobalMean
537185,"Thuner Stadtlauf, Thun",dim. 24.08.2008,2008,8,10-M20,10.0,M,01:00:47.100000,00:06:04,3.0,21.0,Sunny,http://services.datasport.com/2008/lauf/thunstadt,Abbet Damien 1984.0,00:48:47.309882,00:00:16.720411,678,00:49:15.140225,-27.830344
537188,"Thuner Stadtlauf, Thun",dim. 24.08.2008,2008,8,10-M40,10.0,M,00:45:05,00:04:30,3.0,21.0,Sunny,http://services.datasport.com/2008/lauf/thunstadt,Ackermann Thomas 1963.0,00:48:47.309882,00:00:16.720411,678,00:49:15.140225,-27.830344
537195,"Thuner Stadtlauf, Thun",dim. 24.08.2008,2008,8,10-M20,10.0,M,00:44:25.600000,00:04:26,3.0,21.0,Sunny,http://services.datasport.com/2008/lauf/thunstadt,Aeschbacher Thomas 1979.0,00:48:47.309882,00:00:16.720411,678,00:49:15.140225,-27.830344
537198,"Thuner Stadtlauf, Thun",dim. 24.08.2008,2008,8,10-M40,10.0,M,00:46:11.800000,00:04:37,3.0,21.0,Sunny,http://services.datasport.com/2008/lauf/thunstadt,Aeschlimann Martin 1964.0,00:48:47.309882,00:00:16.720411,678,00:49:15.140225,-27.830344
537201,"Thuner Stadtlauf, Thun",dim. 24.08.2008,2008,8,10-M40,10.0,M,00:38:33,00:03:51,3.0,21.0,Sunny,http://services.datasport.com/2008/lauf/thunstadt,Albrecht Beda 1962.0,00:48:47.309882,00:00:16.720411,678,00:49:15.140225,-27.830344
537202,"Thuner Stadtlauf, Thun",dim. 24.08.2008,2008,8,10-M30,10.0,M,00:37:32.300000,00:03:45,3.0,21.0,Sunny,http://services.datasport.com/2008/lauf/thunstadt,Alig Jacobson Danny 1972.0,00:48:47.309882,00:00:16.720411,678,00:49:15.140225,-27.830344
537205,"Thuner Stadtlauf, Thun",dim. 24.08.2008,2008,8,10-M50,10.0,M,00:45:25.100000,00:04:32,3.0,21.0,Sunny,http://services.datasport.com/2008/lauf/thunstadt,Ambühl Eduard 1952.0,00:48:47.309882,00:00:16.720411,678,00:49:15.140225,-27.830344
537208,"Thuner Stadtlauf, Thun",dim. 24.08.2008,2008,8,10-M30,10.0,M,00:57:59,00:05:47,3.0,21.0,Sunny,http://services.datasport.com/2008/lauf/thunstadt,Ammann Urs 1972.0,00:48:47.309882,00:00:16.720411,678,00:49:15.140225,-27.830344
537210,"Thuner Stadtlauf, Thun",dim. 24.08.2008,2008,8,10-M40,10.0,M,00:51:06.700000,00:05:06,3.0,21.0,Sunny,http://services.datasport.com/2008/lauf/thunstadt,Amstutz Martin 1961.0,00:48:47.309882,00:00:16.720411,678,00:49:15.140225,-27.830344
537212,"Thuner Stadtlauf, Thun",dim. 24.08.2008,2008,8,10-M40,10.0,M,00:40:05.500000,00:04:00,3.0,21.0,Sunny,http://services.datasport.com/2008/lauf/thunstadt,Andrade Fernando 1966.0,00:48:47.309882,00:00:16.720411,678,00:49:15.140225,-27.830344


In [77]:
km10.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance', 'Sex',
       'Time', 'Pace', 'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'UserID',
       'MeanRace', 'StdRace', 'Count', 'MeanMultipleYears',
       'DiffFromGlobalMean', 'AvgTemp'],
      dtype='object')

In [92]:
final_data=km10.groupby(km10.RaceID).first().reset_index()
final_data=final_data[['Race', 'RaceYear','Distance',
       'Weather', 'AvgTemp', 'DiffFromGlobalMean',
       'MeanRace', 'MeanMultipleYears',
        'StdRace', 'Count', 'MinTemp', 'MaxTemp','RaceID' ]]
final_data['CatWeather']=final_data.Weather.apply(lambda x: dict_favorable_weather[x])
final_data

,Race,RaceYear,Distance,Weather,AvgTemp,DiffFromGlobalMean,MeanRace,MeanMultipleYears,StdRace,Count,MinTemp,MaxTemp,RaceID,CatWeather
0,"Thuner Stadtlauf, Thun",2008,10.0,Sunny,12.0,-27.830344,00:48:47.309882,00:49:15.140225,00:00:16.720411,678,3.0,21.0,http://services.datasport.com/2008/lauf/thunstadt,2
1,"Hallwilerseelauf, Beinwil am See",2009,10.0,Light rain shower,11.5,-29.087009,00:52:24.505473,00:52:53.592482,00:00:20.490442,676,9.0,14.0,http://services.datasport.com/2009/lauf/hallwil,0
2,20km de Lausanne,2009,10.0,Clear,12.0,-48.763462,00:55:31.464564,00:56:20.228026,00:00:07.646859,4789,6.0,18.0,http://services.datasport.com/2009/lauf/km20,2
3,"Rotseelauf, Ebikon",2009,10.0,Mist,8.0,-0.777238,00:49:43.023642,00:49:43.800880,00:00:16.915312,884,2.0,14.0,http://services.datasport.com/2009/lauf/rotsee,0
4,"Stralugano, Lugano",2009,10.0,Clear,15.5,-95.714648,00:50:42.772580,00:52:18.487228,00:00:15.878698,1116,8.0,23.0,http://services.datasport.com/2009/lauf/stralu...,2
5,"Thuner Stadtlauf, Thun",2009,10.0,Clear,16.0,-37.361370,00:48:37.778855,00:49:15.140225,00:00:18.620667,629,8.0,24.0,http://services.datasport.com/2009/lauf/thunstadt,2
6,"Hallwilerseelauf, Beinwil am See",2010,10.0,Light rain,3.0,-57.439966,00:51:56.152517,00:52:53.592482,00:00:20.593908,735,-1.0,7.0,http://services.datasport.com/2010/lauf/hallwil,0
7,20km de Lausanne,2010,10.0,Patchy rain possible,13.5,38.172897,00:56:58.400923,00:56:20.228026,00:00:07.229416,5197,8.0,19.0,http://services.datasport.com/2010/lauf/km20,1
8,Lausanne Marathon,2010,10.0,Partly cloudy,7.5,-70.397201,00:49:54.288301,00:51:04.685502,00:00:12.126455,1684,3.0,12.0,http://services.datasport.com/2010/lauf/lamara,1
9,"Rotseelauf, Ebikon",2010,10.0,Clear,11.5,-44.150753,00:48:59.650127,00:49:43.800880,00:00:16.664437,786,7.0,16.0,http://services.datasport.com/2010/lauf/rotsee,2


In [94]:
np.corrcoef(final_data.MinTemp,final_data.DiffFromGlobalMean)

array([[ 1.        ,  0.40764921],
       [ 0.40764921,  1.        ]])

In [65]:
compact=km10.groupby(km10.Race).first()
compact=compact[['Distance','MeanRace']]
compact

,Distance,MeanRace
Race,,
20km de Lausanne,10.0,00:55:31.464564
"Hallwilerseelauf, Beinwil am See",10.0,00:52:24.505473
Lausanne Marathon,10.0,00:49:54.288301
"Rotseelauf, Ebikon",10.0,00:49:43.023642
"Stralugano, Lugano",10.0,00:50:42.772580
"Thuner Stadtlauf, Thun",10.0,00:48:47.309882


# Others

In [34]:
la20=full_df[full_df.Race=='Schweizer Frauenlauf Bern']

In [ ]:
la20.Distance.value_counts()

In [ ]:
la20=la20[la20.Distance==10.000]

In [ ]:
la20.shape

In [ ]:
la20.RaceYear.value_counts()

In [ ]:
def add_mean_std_race(x):
    x['MeanRace']=np.mean(x.Time)
    x['StdRace']=np.std(x.Time)/np.sqrt(x.shape[0])
    x['Count']=x.shape[0]
    return x

In [ ]:
la20=la20.groupby(la20.RaceYear).apply(add_mean_std_race)

In [ ]:
years=[int(x) for x in la20.RaceYear.value_counts().index]
years.sort()

In [ ]:
dict_compare_times={years[0]:1}

In [ ]:
for year in years[1:]:
    db=la20[(la20.RaceYear==year) | (la20.RaceYear==year-1)]
    
    gr=db.groupby(db.UserID).filter(lambda x: len(x)>1)
    gr1=gr[gr.RaceYear==year-1]
    gr2=gr[gr.RaceYear==year]
    
    mean1=np.mean(gr1.Time)
    mean2=np.mean(gr2.Time)
    factor=mean2/mean1
    print(factor)
    dict_compare_times[year]=dict_compare_times[year-1]*factor

In [ ]:
mean1=np.mean(la20[la20.RaceYear==years[0]].Time)

In [ ]:
def add_comparison_time(x):
    year=x.iloc[0].RaceYear
    x['StandardizedTime']=dict_compare_times[year]*mean1
    return x

In [ ]:
la20=la20.groupby(la20.RaceYear).apply(add_comparison_time)

In [ ]:
la20['AvgTemp']=(la20.MinTemp+la20.MaxTemp)/2

In [ ]:
stats=la20[['Race','RaceYear','Distance','Weather','AvgTemp','StandardizedTime','MeanRace','StdRace','Count']].groupby(la20.RaceYear).first()

In [ ]:
stats

In [ ]:
stats

# Analysis of men

In [ ]:
men_full_df=full_df[full_df.Sex=='M']

In [ ]:
men_marathon=men_full_df[men_full_df.Distance==42.195]

In [ ]:
men_marathon.shape

In [ ]:
men_marathon.head(3)

In [ ]:
def add_mean_std_race(x):
    x['MeanRace']=np.mean(x.Time)
    x['StdRace']=np.std(x.Time)/np.sqrt(x.shape[0])
    x['Count']=x.shape[0]
    return x

In [ ]:
men_marathon=men_marathon.groupby(men_marathon.RaceID).apply(add_mean_std_race)

In [ ]:
men_marathon_means=men_marathon.groupby(men_marathon.RaceID).first()

In [ ]:
men_marathon_means.head(1)

In [ ]:
men_marathon_means=men_marathon_means[['Race','Date','Distance','MinTemp','MaxTemp','Weather','MeanRace','StdRace','Count']]

In [ ]:
men_marathon_means['AvgTemp']=(men_marathon_means.MinTemp+men_marathon_means.MaxTemp)/2

In [ ]:
men_marathon_means=men_marathon_means[['Race','Date','Distance','Weather','AvgTemp','MeanRace','StdRace','Count']]

In [ ]:
men_marathon_means

In [ ]:
men_marathon_means.Weather.value_counts()

# Classify different marathons

In [ ]:
gr_means=men_marathon_means.groupby(men_marathon_means.Race)

In [ ]:
def add_place_mean(x):
    x['MeanPlace']=np.mean(x.MeanRace)
    x['CountPlace']=np.mean(x.Count)
    x['NumberOfRaces']=x.shape[0]
    return x

In [ ]:
place_mean=gr_means.apply(add_place_mean)

In [ ]:
place_mean=place_mean.groupby(men_marathon_means.Race).first()
place_mean=place_mean[['Distance','MeanPlace','CountPlace','NumberOfRaces']]

In [ ]:
place_mean

As you can see, a lot of marathon have too few partecipants to be statistically significative.
Moreover, the Interlaken and Zermatt marathon are slow-pace marathon, for the track chosen (with a lot of uphill).
We exclude these competitions from our analysis

The marathons to analyse are:
- Lausanne Marathon
- Lucerne Marathon, Luzern
- SwissCityMarathon - Lucerne, Luzern
- Zürich Marathon, Teamrun und Cityrun

# Analyse selected runs

In [ ]:
final_marathons=men_marathon_means[(men_marathon_means.Race=='Lausanne Marathon') | 
                                   (men_marathon_means.Race=='Lucerne Marathon, Luzern') |
                                   (men_marathon_means.Race=='SwissCityMarathon - Lucerne, Luzern') |
                                   (men_marathon_means.Race=='Zürich Marathon, Teamrun und Cityrun')]

In [ ]:
final_marathons[final_marathons.Race=='Zürich Marathon, Teamrun und Cityrun']